<a href="https://colab.research.google.com/github/saito828koki/engagement_auto_recorder/blob/master/auto_insights_recorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread requests oauth2client pandas

In [ ]:
access_token = 'アクセストークン'

ig_user_id = 'インスタのビジネスアカウントのID'

sheet_name = '記録するスプレッドシートの名前'

post_num = 30 # 直近の何投稿のデータを取ってくるかのパラメータ。最大でも150程度までにしておかないとAPIの制限に引っかかる。

In [ ]:
import requests
import json
import datetime
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe

In [ ]:
def get_post_data(access_token, ig_user_id, post_num):
  params = (
    ('access_token', access_token),
    ('fields', 'id, shortcode'),
  )
  media_response = requests.get('https://graph.facebook.com/v8.0/' + ig_user_id + '/media', params=params)  
  media_dict = media_response.json()
  media_id = []
  media_url = []
  for num, d in enumerate(media_dict["data"]):
    if num >= 100:
      break
    media_id.append(d["id"])
    media_url.append("https://www.instagram.com/p/" + d["shortcode"])
  params = (
    ('access_token', access_token),
    ('metric', 'engagement,impressions,reach,saved'),
  )

  insights_list = []

  for m in media_id:
    insights_response = requests.get('https://graph.facebook.com/v8.0/' + m + '/insights', params=params)
    insights_dict = insights_response.json()
    add_dict = {}
    for d in insights_dict["data"]:
      add_dict[d["name"]] = d["values"][0]["value"]
    insights_list.append(add_dict)
  return media_url, insights_list


sheet_num_item_dict = {0: 'engagement', 1: 'impressions', 2: 'reach', 3: 'saved'}

def main(sheet_name, sheet_num, media_url, insights_list):
  # 認証処理
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())

  # sheet_nameに渡された名前のスプレッドシートをopen
  # 存在しないとエラーになる
  worksheet = gc.open(sheet_name).get_worksheet(sheet_num)

  # List[List]の形でworksheetを読み込む
  df = worksheet.get_all_values()
  # sheetが空のときは投稿のURLをカラムとして初期化する
  if df == []:
    tmp = ['']
    for p in media_url:
      tmp.append(p)
    df.append(tmp)
  
  else:
    for p in media_url:
      # 今までクロールしたことのある投稿かチェック
      if p not in df[0]:
        #なければカラムを足す
        df[0].append(p)
        # 今までの記録があれば空欄で初期化する操作もする
        if len(df) > 1:
          for i in range(1, len(df)):
            df[i].append('')
    

  add_row = [''] * len(df[0])
  # indexには今日の日付を使用する
  today = datetime.date.today()
  add_row[0] = today
  for i, p in enumerate(df[0][1:]):
    if p in media_url:
      add_row[i+1] = insights_list[media_url.index(p)][sheet_num_item_dict[sheet_num]]
  df.append(add_row)

  set_with_dataframe(worksheet, pd.DataFrame(df), include_index=False, include_column_header=False)

In [ ]:
media_url, insights_list = get_post_data(access_token, ig_user_id)
for i in range(4):
  main(sheet_name, i, media_url, insights_list)